# <font color = "slateblue"> EXAM II </font>
#### <font color = "slateblue"> Math and Stats</font>
#### <font color = "slateblue"> Date: 21st, Feb 2021 </font>

---



## <font color = "limegreen">Student Data</font>

Fill your **NAME** only:

#### Student Name: ..........BEA...................

---


## <font color = "limegreen">Instructions</font>

Read carefully these instructions and follow them during your quiz and in your submission.

 * The exam lasts **1 week**
 * Read carefully the questions and do not answer before knowing what is asked
 * Full marks require **full explanations**. Just answering the question is not enough, for example, if one answer is that the type of data is *panel data*, just saying that will not grant you more than the 25% of the available points.
 * The **answers** must be written right below the questions made in this notebook. Use Code and Text cells as needed


---

## <font color = "limegreen"> Packages </font>

In the next cell code add **ALL** the modules you will use in your exam: `numpy`, `pandas`,... 

In [1]:
#%load_ext nb_black
import numpy as np
import pandas as pd
import scipy.stats as ss
import statsmodels.stats.weightstats as smw
import statsmodels.stats.proportion as smp
import matplotlib.pyplot as plt
import math

plt.style.use("seaborn")

In case you work with colab notebooks, use the following cell to connect to your drive

In [2]:
# not necessary

---

## <font color = "limegreen">The Data </font>

**The High School ad Beyond study contains a set of 200 observations on high school seniors. The variables we find in the dataset are**

<br>

| Variable    |           Description           |
|-------------|---------------------------------|
|  id    | Student ID |
|  gender  | Student's gender (female, male) |
|  race    | Student's race (african american, asian, hispanic, white) |
|  ses  | Socio economic status of student's family (low, middle, high) |
| schtyp | Type of school (public, private) | 
| prog  | Type of program (general, academic, vocational)|
| read  | Standardized reading score |
| write | Standardized writing score |
| math  | Standardized math score |
| science | Standardized science score |
| socst | Standardized social studies score|

<br>

**in order to load it, use the next *code cell*, taking into account that it is a .txt file (named `hsb2` which can be found in the folder of datasets). Add in this same code cell a instruction that lets you see the 5 first rows of the dataset.**

In [3]:
hsb2 = pd.read_csv("hsb2.txt", sep="\t", index_col="id")
hsb2.head()

,gender,race,ses,schtyp,prog,read,write,math,science,socst
id,,,,,,,,,,
70,male,white,low,public,general,57,52,41,47,57
121,female,white,middle,public,vocational,68,59,53,63,61
86,male,white,high,public,general,44,33,54,58,31
141,male,white,high,public,vocational,63,44,47,53,56
172,male,white,middle,public,academic,47,52,57,53,61


In [4]:
print("We have", hsb2.shape[0], "samples available")

We have 200 samples available


In [5]:
hsb2.describe()

,read,write,math,science,socst
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,52.230000,52.775000,52.645000,51.850000,52.405000
std,10.252937,9.478586,9.368448,9.900891,10.735793
min,28.000000,31.000000,33.000000,26.000000,26.000000
25%,44.000000,45.750000,45.000000,44.000000,46.000000
50%,50.000000,54.000000,52.000000,53.000000,52.000000
75%,60.000000,60.000000,59.000000,58.000000,61.000000
max,76.000000,67.000000,75.000000,74.000000,71.000000



---

## Questions

## 1.- <font color = "Red"> Estimation </font>

**In this first part, let's focus in confidence intervals**

### 1.1.- <font color = "Blue">Average Scores (2 Points)</font>

**Find the 95% confidence intervals for the standarized scores of all the variables. Give an interpretation of them.**

In [6]:
# first, we define common values for all the distributions
CL = 0.95
SL = 1 - CL
zcrit = ss.norm.isf(SL / 2) # we compute zcrit using the inverse survival function at alpha/2 
                            # because sigmas for our populations are known

# then we select the variables we are interested in and create a for loop
for var in hsb2.loc[:, "read":].columns:
    # obtain var information
    var_mean = hsb2[var].mean()
    var_stdev = hsb2[var].std()
    var_n = len(hsb2[var])

    # calculate confidence interval
    ll = var_mean - zcrit * var_stdev / np.sqrt(var_n)
    ul = var_mean + zcrit * var_stdev / np.sqrt(var_n)

    # print the results
    print("The {:.0%} confidence interval for {} is [{:4.2f}, {:4.2f}]".format(CL,var,ll,ul))

The 95% confidence interval for read is [50.81, 53.65]
The 95% confidence interval for write is [51.46, 54.09]
The 95% confidence interval for math is [51.35, 53.94]
The 95% confidence interval for science is [50.48, 53.22]
The 95% confidence interval for socst is [50.92, 53.89]


The calculated confidence intervals allow us to affirm that we are 95% confident that the actual population mean of each variable will be included in said intervals.

### 1.2.- <font color = "Blue">Comparison of Populations (1 Point)</font>

**Using these single population scores, would you say that there is a significant difference bewteen the average grades in `math` and in `read`? Why?**

In [7]:
print("The mean of math is","{:4.2f}".format(hsb2["math"].mean()),
    "and its standard deviation is","{:4.2f}".format(hsb2["math"].std()))
print("The mean of read is","{:4.2f}".format(hsb2["read"].mean()),
    "and its standard deviation is","{:4.2f}".format(hsb2["read"].std()))

The mean of math is 52.65 and its standard deviation is 9.37
The mean of read is 52.23 and its standard deviation is 10.25


Given the fact that both means, standard deviations, and 95% confidence intervals are very close, we could affirm there is no significant difference between both variables.

### 1.3.- <font color = "Blue">Comparison of Populations (2 Points)</font>

**Using specific methods for two populations, find the 90% confidence interval for the difference in the average grades of `math` and `read`. Is this result in agreement with your previous argument?**

In order to calculate the difference in grades we create a new variable

In [8]:
hsb2["diff_mr"] = hsb2.math - hsb2.read
hsb2.diff_mr.head()

id
70    -16
121   -15
86     10
141   -16
172    10
Name: diff_mr, dtype: int64

In [9]:
# instead of using the zcrit formula as before, we are going to use statsmodels now
CL = 0.90
SL = 1 - CL

# we calculate the descriptive stats
des = smw.DescrStatsW(hsb2["diff_mr"])

# and obtain the lower and upper limits of the confidence interval
ll, ul = des.zconfint_mean(SL)  # the number of available samples is > 40 so we use zconfint

# print the results
print("The {:.0%} confidence interval for".format(CL),"for the difference is [{:4.2f}, {:4.2f}]".format(ll, ul))

The 90% confidence interval for for the difference is [-0.53, 1.36]


Given the fact that 0 is included in the 90% confidence interval of the difference, we can affirm we are 90% confident there is no significant difference between both variables, which is in line with what we expected in point 1.2.

### 1.4.- <font color = "Blue">Proportions (1 Point)</font>

**Find the confidence interval for the proportion of males and of females (separatedly) that failed in `science`**

We need to create a dataset with the samples where `science` was failed.

In [10]:
science_fail = hsb2[hsb2["science"] < 50]
science_fail.head()

,gender,race,ses,schtyp,prog,read,write,math,science,socst,diff_mr
id,,,,,,,,,,,
70,male,white,low,public,general,57,52,41,47,57,-16
11,male,hispanic,middle,public,academic,34,46,45,39,36,11
38,male,african american,low,public,academic,45,57,50,31,56,5
12,male,hispanic,middle,public,vocational,37,44,45,39,46,8
53,male,african american,middle,public,vocational,34,37,46,39,31,12


We can now compute the required intervals for males and females

In [12]:
# First we set a confidence level 
CL = 0.95
SL = 1 - CL

for gender in science_fail.gender.unique():
    # we calculate the proportion of failed science vs total attempts for each gender
    n_gen = len(hsb2[hsb2.gender == gender].science)  # total number of individuals of the selected gender in our sample
    x_gen = len(science_fail[science_fail.gender == gender].science)  # total number of individuals of the selected gender who failed science

    # we can now calculate p_hat for each gender
    p_hat_gen = x_gen / n_gen

    # next we calculate np and nq of the binomial distribution and print them
    n_p = n_gen * p_hat_gen
    n_q = n_gen * (1 - p_hat_gen)
    print("The values for the normality approximation are, for {}s, {:2.0f} and {:2.0f}.".format(gender,n_p,n_q))

    # since the calculated values of np and nq are both > 10, we can use the normal approximation to the binomial distribution
    
    # then we compute zcrit 
    zcrit = ss.norm.isf(SL / 2)

    # and now we can obtain the lower and upper limits of our confidence interval
    ll = p_hat_gen - zcrit * math.sqrt(p_hat_gen * (1 - p_hat_gen) / n_gen)
    ul = p_hat_gen + zcrit * math.sqrt(p_hat_gen * (1 - p_hat_gen) / n_gen)

    print("The {:.0%} CI for the proportion of {}s who failed science is [{:.2%}, {:.2%}]".format(CL, gender, ll, ul),"\n")

The values for the normality approximation are, for males, 30 and 61.
The 95% CI for the proportion of males who failed science is [23.31%, 42.63%] 

The values for the normality approximation are, for females, 41 and 68.
The 95% CI for the proportion of females who failed science is [28.52%, 46.71%] 



## 2.- <font color = "Red"> Hypothesis Testing </font>

**For each of the following questions clearly state the decision scheme you choose and why. Also, before any test, check that the conditions for inference are satisfied.**

### 2.1.- <font color = "Blue"> First Test (2 Points) </font> 

**Perform a test to see if the average grade in `math` is above 52 points. What about the average grade in `science`, is it greater than 52?**

The conditions of inference for these tests are:
- *Independence*: we assume that our sample students were honest and didn't copy
- *Normality*: check if we find the following:<ul><li>
n > 40 - the number of samples in both `math`and `science` are above 40
<li>no extreme outliers - grades are mandatorly between 0 and 100, so there cannot be any values beyond those limits. And anyway, checking max and min on each variable result in values away from the theoretical max and min.

Our decision scheme is as follows:
\begin{equation}
H_0:\{\text{average below 52}\},\quad H_1:\{\text{average above 52}\}
\end{equation}

so we must perform a right-tailed test with $\mu_0$ = 52:
\begin{equation}
H_0:\{\mu\leq 52\}, \quad H_1:\{\mu > 52\}
\end{equation}

In [13]:
# Data preparation for our test
mu0 = 52
SL = 0.05
n = len(hsb2.math) # >40, therefore we can use the normal approximation with the same standard deviation

# Descriptive Stats
xmean = hsb2.math.mean()
stdev = hsb2.math.std()

# Calculate z-statistic and print it
zstat = (xmean - mu0)/(stdev/np.sqrt(n))
print('z-statistic: ', round(zstat,2))

# Calculate z-critical and print it
zcrit = ss.norm.isf(SL)
print('z-critical:', round(zcrit,2))

# Compare both values to perform our test
if (zstat > zcrit):
  print('Reject the NULL hypothesis')
else:
  print('Fail to Reject the NULL hypothesis')

z-statistic:  0.97
z-critical: 1.64
Fail to Reject the NULL hypothesis


Since we Fail to Reject the NULL hypothesis, we do not find evidence against $H_0$ for a 5% of significance level. Then we cannot say that the average grade in `math` is above 52

In [14]:
# Regarding science, we are going to use statsmodels for the calculations

# Data
mu0 = 52
SL = 0.05

# Descriptive Stats
des_f = smw.DescrStatsW(hsb2.science)

# Calculate z-statistic and print it
zstat, pval = des_f.ztest_mean(value = mu0, alternative='larger') # ztest because n > 40
print('z-statistic: ', round(zstat,2))

# Calculate z-critical and print it
zcrit = ss.norm.isf(SL)
print('z-critical:', round(zcrit,2))

# Compare both values to perform our test
if (zstat > zcrit):
  print('Reject the NULL hypothesis')
else:
  print('Fail to Reject the NULL hypothesis')

z-statistic:  -0.21
z-critical: 1.64
Fail to Reject the NULL hypothesis


Again, since we Fail to Reject the NULL hypothesis, we do not find evidence against $H_0$ for a 5% of significance level. Then we cannot say that the average grade in `science` is above 52

### 2.2.- <font color = "Blue">Second Test (2 Points)</font>

**Perform a test to see if the average grade in `science` and in `write` is different between males and females (separatedly the different courses)**

The conditions of inference for these tests are:
- *Independence*: we assume that our sample students were honest and didn't copy
- *Normality*: check if we find the following:<ul><li>
n > 40 - the number of samples in both `math`and `science` are above 40
<li>no extreme outliers - grades are mandatorly between 0 and 100, so there cannot be any values beyond those limits. And anyway, checking max and min on each variable result in values away from the theoretical max and min.


Our decision scheme is as follows:
\begin{equation}
H_0:\{\text{averages are equal}\},\quad H_1:\{\text{averages are not equal}\}
\end{equation}

so we must perform a two-tailed test:
\begin{equation}
H_0:\{\mu_1 = \mu_2\}, \quad H_1:\{\mu_1 \neq \mu_2\}
\end{equation}

which can be written as:
\begin{equation}
H_0:\{\mu_1 - \mu_2 = 0\}, \quad H_1:\{\mu_1 - \mu_2 \neq 0\}
\end{equation}


In [ ]:
# Data
SL = 0.05

# nested for loops to go over variables and genders to compare 
for var in ("science","write"):
    subvars = {}

    for gender in ("male","female"):
        subvars[gender] = hsb2.loc[hsb2.gender == gender,var]
    
    # Test that the two-sided difference in the mean is 0
    zstat, pval = smw.ztest(subvars["male"], subvars["female"], value = 0, alternative='two-sided') 

    print("The calculated p-value for {} is {:.4f}, that we compare to the SL and obtain that we must ".format(var,pval),end="")
    if (pval < SL):
        print("REJECT the NULL hypothesis.")
    else:
        print("FAIL to REJECT the NULL hypothesis.")

We fail to reject the NULL hypothesis for the `science` variable, therefore we do not find evidence against $H_0$ for a 5% of significance level. Then we cannot say that the average grade in `science` for males and females is different.<br>
On the contrary, since we reject the NULL hypothesis for the `write` variable, it means that we find evidence against $H_0$ for a 5% of significance level. Then we can say that the average grade in `write` for males and females is different.

### 2.3.- <font color = "Blue"> Third Test (2 Points)</font>

**Perform a test to see if the proportion of students from public schools who passed `socst` is the same as that of those from a private school.**

The conditions of inference for these tests are:
- *Independence*: we assume that our sample students were honest and didn't copy
- *Normality*: check if we find the following:<ul><li>
n > 40 - the number of samples in both `math`and `science` are above 40
<li>no extreme outliers - grades are mandatorly between 0 and 100, so there cannot be any values beyond those limits. And anyway, checking max and min on each variable result in values away from the theoretical max and min.
<li>since we are comparing two populations, we must also check the success-failure condition. See below

Our decision scheme is as follows:
\begin{equation}
H_0:\{\text{proportions are equal}\},\quad H_1:\{\text{proportions are not equal}\}
\end{equation}

so we must perform a two-tailed test:
\begin{equation}
H_0:\{P_{pub} = P_{pri}\}, \quad H_1:\{P_{pub} \neq P_{pri}\}
\end{equation}

which can be written as:
\begin{equation}
H_0:\{P_{pub} - P_{pri} = 0\}, \quad H_1:\{P_{pub} - P_{pri} \neq 0\}
\end{equation}


In [ ]:
# Following the reasoning applied in point 1.4 we first create a dataframe with those 
# individuals who passed socst, and then calculate proportions
socst_pass = hsb2[hsb2["socst"] >= 50]
socst_pass.head()

In [ ]:
# Data
SL = 0.05

n_school = {}
x_school = {}
# for loop to go over types of schools 
for school in ("public","private"):
    # calculate the proportions
    n_school[school] = len(hsb2.loc[hsb2.schtyp == school].socst) # total number of individuals of the selected type of school in our sample
    x_school[school] = len(socst_pass[socst_pass.schtyp == school].socst) # number of individuals of the selected type of school who passed socst

    # we can now calculate p_hat for each type of school
    p_hat_school = x_school[school] / n_school[school]

    # next we calculate np and nq of the binomial distribution and print them
    n_p = n_school[school] * p_hat_school
    n_q = n_school[school] * (1 - p_hat_school)
    print("The values for the normality approximation are, for {} schools, {:.0f} and {:.0f}.".format(school,n_p,n_q))

It seems like we shouldn't go on and use the normal approximation, but since the success-failure condition of np and nq > 10 is a rule of thumb and some textbooks place the threshold at 5, we continue. Also, because if we use the p_pooled proportion now all np and nq are above ten:

In [ ]:
p_pooled = len(socst_pass) / len(hsb2)
for school in ("public","private"):
    n_p = n_school[school] * p_pooled
    n_q = n_school[school] * (1 - p_pooled)
    print("The values for the normality approximation are, for {} schools, using the p-pooled proportion: {:.0f} and {:.0f}.".format(school,n_p,n_q))

We can now go on with confidence that using a normal approximation is correct

In [ ]:
n = np.array(list(n_school.values()))
x = np.array(list(x_school.values()))
    
# Test that the two-sided difference in the proportions is 0
zstat, pval = smp.proportions_ztest(x, n, value = 0, alternative='two-sided') 
print("")
print("The calculated p-value is {:.4f}, that we compare to the SL and obtain that we must ".format(pval),end="")
if (pval < SL):
    print("REJECT the NULL hypothesis.")
else:
    print("FAIL to REJECT the NULL hypothesis.")

We fail to reject the null hypothesis, therefore we we cannot say that the proportions of students who passed social studies in public and private schools are different.

### 2.4.- <font color = "Blue"> Power (2 Points)</font>

**You have already performed a test to compare the grades in `science` between males and females. Whatever your previous result, you know that an average difference of 3 points may be indicative of a case to analyse deeply. Then find the probability that you detect a difference of 3 points from your sample in favor a males in case this difference actually exists.**

For this averages of two population comparison we are going to make some assumptions for the null hypothesys:
- both populations are of the same size
- both pupulations have the same mean
- both populations have the same standard deviation

so that we can simplify our hypothesis testing.

Our decision scheme is as follows:
\begin{equation}
H_0:\{\mu_1 = \mu_2\}, \quad H_1:\{\mu_1 \neq \mu_2\}
\end{equation}

which can be written as:
\begin{equation}
H_0:\{\mu_{males} - \mu_{females} = 0\}, \quad H_1:\{\mu_{males} - \mu_{females} = 3\}
\end{equation}


In [15]:
# significance level
SL = 0.05

# population means
mu0 = 0
mu1 = 3

# sample values
mean = hsb2['science'].mean()
stdev = hsb2['science'].std()
n = len(hsb2['science'])

# critical t
tcrit = ss.t.isf(SL, n-1)

# effect size
delta = mu0 - mu1

# beta value
beta_val = delta / (stdev/(np.sqrt(n))) + tcrit

# power of the test
beta = ss.t.cdf(beta_val, n-1)
power = 1 - beta

# print the output
print('The probability that if the difference in average were 3 we would detect it is {:.2%}'.format(power))


The probability that if the difference in average were 3 we would detect it is 99.54%


### 2.5.- <font color = "Blue"> Sample size (2 Points)</font>

**If in the previous question you want that the probability is 99.999% while the significance level is 0.001, what is the sample size you may need?**

We maintain our simplification and use the normal approximation for the calculation

In [ ]:
# we maintain the significance level and look for the new n
searched_power = .999
searched_beta = 1 - searched_power

A = ss.norm.ppf(searched_beta)

# critical z
zcrit = ss.norm.isf(SL)

# sample size needed
new_n = ((A - zcrit) * stdev / delta)**2

# print the output
print('The sample size needed is {}'.format(int(np.round(new_n,0))))